# Installing Dependencies

In [148]:
import numpy as np # zero matrices, arrays, .max(), linspace
from numpy.random import randint # random integers

# Cleaning Up Matrix For Printing

The first function add_bases() employs three parameters (x = first sequence, y = second sequence, m = current matrix). This function adds genetic sequences on the side of the row and column to allow for easier formatting.

Before implementing add_bases():
![unfilled_version.png](Unfilled_version.png)

After implementing add_bases()
![filled_version.png](filled_version.png)

The second function print_matrix() has one parameter (matrix = current matrix). It employs methods to allow for better spacing and line formatting for the array itself (evident in second picture).

In [118]:
def add_bases (x, y, m):
    matrix = m.tolist()
    matrix.insert(0, [" ",  " "]+list(y)) # Adding the genetic sequence to the top row
    for i in range(2, len(x) + 2): # Adding the genetic sequence to the leftmost column
        matrix[i] = [list(x)[i-2]] + matrix[i]
    matrix[1] = [" "] + matrix[1]
    return matrix
    
def print_matrix(matrix):
    s = [[str(e) for e in row] for row in matrix]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print('\n'.join(table))

# Needleman-Wunsch Algorithm

The Needleman-Wunsch algorithm is run through the nw() function with four parameters (x: the first sequence, y: the second sequence, mismatch: the penalty for mismatching two sequences, gap: the penalty for an insertion/deletion).

Global alignment refers to trying to get every element inside two particular genetic sequences to align with one another; this is often used to calculate evolutionary processes that were acted to preserve biological function through changing a specific codon/nucleotide (not by random chance). The main purpose of the Needleman-Wunsch algorithm is to discover how one sequence can relate to another with mutations (change in nitrogenous bases), additions, or deletions of nitrogenous base pairs.

Global alignment:
![global_alignment.png](global_alignment.png)

For example, the algorithm would identify the sequences “CGC” and “ACGC” to be evolutionarily related, with a simple addition of adenine to get “CGC” from “ACGC”. Of course, for more complex sequences, a more optimal algorithm is necessary.

The main goal of the N-W algorithm is to minimize the number of events to explain the differences between two nucleotide sequences, by running this algorithm from exponential to quadratic runtime using dynamic programming. It would recursively fill up a table, discover optimal scores, then trace back the optimal score to then find the optimal solution to the code. This is done through a base matrix (which simply runs forward the dynamic programming algorithm) and then a subsequent pointer matrix (which tries to run back into the code to figure out the most optimal set of mutations to arrive from one sequence to another).

Dynamic programming process:
![needleman_wunsch_intuition.png](needleman_wunsch_intuition.png)

Article explaining this in depth: https://studentsxstudents.com/coding-global-sequence-alignment-using-the-needleman-wunsch-algorithm-d47971ebbe5

In [154]:
def nw(x, y, match = 1, mismatch = 1, gap = 2):
    nx = len(x) # length of first base sequence
    ny = len(y) # length of second base sequence
    
    # Initialization process - forming the base matrix
    F = np.zeros((nx + 1, ny + 1)) # an array 
    F[:,0] = np.linspace(0, -gap*nx, nx + 1)
    F[0,:] = np.linspace(0, -gap*ny, ny + 1)
    
    # Pointers to trace through an optimal aligment.
    P = np.zeros((nx + 1, ny + 1))
    P[:,0] = 3
    P[0,:] = 4
    
    print(F)
    print(P)
    # Temporary scores for aligning base x and base y.
    t = np.zeros(3)
    for i in range(nx):
        for j in range(ny):
            
            # Iteration step: take the max (inserting gap in first sequence, inserting gap in second sequence, match or mutation)
            if x[i] == y[j]:
                t[0] = F[i,j] + match
            else:
                t[0] = F[i,j] - mismatch
            
            # Inserting gap in first sequence
            t[1] = F[i,j+1] - gap
            # Inserting gap in second sequence
            t[2] = F[i+1,j] - gap
            tmax = np.max(t)
            
            F[i+1,j+1] = tmax
            if t[0] == tmax:
                P[i+1,j+1] += 2
                
            # Higher weights for inserting gaps rather than matches/mismatches
            if t[1] == tmax:
                P[i+1,j+1] += 3
            if t[2] == tmax:
                P[i+1,j+1] += 4

    # Trace through an optimal alignment.
    i = nx
    j = ny
    rx = []
    ry = []
    
    tracer_matrix = np.zeros((nx+1, ny+1))
    while i > 0 or j > 0:
        tracer_matrix[i, j] = -1
        
        if P[i,j] in [2, 5, 6, 9]:
            rx.append(x[i-1])
            ry.append(y[j-1])
            
            i -= 1
            j -= 1
        
        # if there's a gap in the first sequence
        elif P[i,j] in [3, 7]:
            rx.append(x[i-1])
            ry.append('-')
            i -= 1
            
        # if there's a gap in the second sequence
        elif P[i,j] in [4]:
            rx.append('-')
            ry.append(y[j-1])
            j -= 1
    
    
    print("Pointer matrix:")
    pointer_matrix = add_bases(x, y, P)
    print_matrix(pointer_matrix)
    
    print()
    print("Tracer matrix:")
    tracer_matrix = add_bases(x, y, tracer_matrix)
    print_matrix(tracer_matrix)
    
    # Reverse the strings.
    print()
    print("Final result:")
    
    rx = ''.join(rx)[::-1]
    ry = ''.join(ry)[::-1]
    
    px = "Sequence 1: " + rx
    py = "Sequence 2: " + ry
    return ['\n'.join([px, py]), rx, ry]

# Random Sequences

This section of the code randomly chooses As, Ts, Cs, and Gs in order to develop a comprehensive understanding of how randomness plays a role in the N-W algorithm. As evidenced by the code below, it seems that randomness would simply confuse the N-W algorithm since there is no optimal mutation path to get from one sequence to another.

In [155]:
np.random.seed(12)

# For random sequences (sequence alignment not optimal)
x = np.random.choice(['A', 'T', 'G', 'C'], 4)
y = np.random.choice(['A', 'T', 'G', 'C'], 4)

printseq, seq1, seq2 = nw(x,y)
print(printseq)

[[ 0. -2. -4. -6. -8.]
 [-2.  0.  0.  0.  0.]
 [-4.  0.  0.  0.  0.]
 [-6.  0.  0.  0.  0.]
 [-8.  0.  0.  0.  0.]]
[[4. 4. 4. 4. 4.]
 [3. 0. 0. 0. 0.]
 [3. 0. 0. 0. 0.]
 [3. 0. 0. 0. 0.]
 [3. 0. 0. 0. 0.]]
Pointer matrix:
 	   	T  	G  	C  	C  
 	4.0	4.0	4.0	4.0	4.0
C	3.0	2.0	6.0	2.0	6.0
C	3.0	5.0	2.0	2.0	2.0
G	3.0	5.0	2.0	2.0	2.0
T	3.0	2.0	3.0	2.0	2.0

Tracer matrix:
 	   	T   	G   	C   	C   
 	0.0	0.0 	0.0 	0.0 	0.0 
C	0.0	-1.0	0.0 	0.0 	0.0 
C	0.0	0.0 	-1.0	0.0 	0.0 
G	0.0	0.0 	0.0 	-1.0	0.0 
T	0.0	0.0 	0.0 	0.0 	-1.0

Final result:
Sequence 1: CCGT
Sequence 2: TGCC


# With E.Coli Data

There is an evident use case in utilizing the Needleman-Wunsch algorithm in finding differences/similarities in parts of the E. coli genetic sequence.

Here, we can discover the exact set of mutations, additions, and deletions that allowed us to arrive from one sequence to another. If you’re interested in accessing the data, the text file “e_coli_seq.txt” should provide the necessary genetic info to test out different E. coli sequences.

Below is the code testing out different E. coli sequences with one another to check for mutations and alterations.

In [153]:
dna = open("e_coli_seq.txt")
query = open("query.txt")

seq_arrange = []
for line in dna:
    if line[0] != '>':
        end = len(line)
        seq_arrange.append(line[:-1])

In [151]:
seq_arrange = [i[:13] for i in seq_arrange]

first_index = 0
second_index = 0

while (first_index == second_index):
    first_index = randint(0, len(seq_arrange))
    second_index = randint(0, len(seq_arrange))
first_seq = seq_arrange[first_index]
second_seq = seq_arrange[second_index]

In [152]:
printseq, seq1, seq2 = nw(first_seq, second_seq)
print(printseq)

Pointer matrix:
 	   	C  	T  	C  	T  	C  	A  	T  	G  	G  	A  	A  	G  	T  
 	4.0	4.0	4.0	4.0	4.0	4.0	4.0	4.0	4.0	4.0	4.0	4.0	4.0	4.0
C	3.0	2.0	4.0	6.0	4.0	6.0	4.0	4.0	4.0	4.0	4.0	4.0	4.0	4.0
C	3.0	5.0	2.0	2.0	4.0	6.0	4.0	4.0	4.0	4.0	4.0	4.0	4.0	4.0
G	3.0	3.0	5.0	2.0	2.0	6.0	6.0	6.0	2.0	6.0	4.0	4.0	6.0	4.0
G	3.0	3.0	5.0	5.0	2.0	2.0	6.0	6.0	2.0	2.0	4.0	4.0	6.0	4.0
T	3.0	3.0	2.0	5.0	2.0	2.0	2.0	2.0	4.0	6.0	2.0	6.0	6.0	2.0
T	3.0	3.0	5.0	2.0	5.0	2.0	2.0	2.0	6.0	6.0	6.0	2.0	6.0	2.0
A	3.0	3.0	3.0	5.0	5.0	5.0	2.0	7.0	2.0	6.0	2.0	6.0	6.0	6.0
T	3.0	3.0	5.0	5.0	2.0	5.0	3.0	2.0	4.0	2.0	6.0	2.0	6.0	2.0
T	3.0	3.0	5.0	5.0	5.0	2.0	3.0	5.0	2.0	6.0	2.0	6.0	2.0	2.0
G	3.0	3.0	3.0	5.0	3.0	5.0	5.0	3.0	2.0	2.0	4.0	4.0	2.0	6.0
A	3.0	3.0	3.0	5.0	3.0	5.0	2.0	3.0	3.0	5.0	2.0	6.0	4.0	4.0
G	3.0	3.0	3.0	5.0	3.0	5.0	3.0	5.0	5.0	2.0	3.0	2.0	2.0	4.0
A	3.0	3.0	3.0	5.0	3.0	5.0	5.0	5.0	3.0	3.0	2.0	2.0	2.0	2.0

Tracer matrix:
 	    	C   	T   	C   	T   	C   	A   	T   	G   	G   	A   	A   	G   	T   
 	0.0 	0.0 	0.0 	0.0 	0.0 